In [ ]:
import os
os.chdir("..")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from TravelAndMutate import trees

In [ ]:
def rewriteMutations(series):
    return [el.replace(" ","").replace("'","").replace("[","").replace("]","") for el in series]
	
def getId(series):
    mapping = {}
    to_return = []
    for el in series:
        string = el.replace("[","").replace("]","").replace("'","").replace(", ","")
        if string in mapping.keys():
            to_return.append(mapping[string])
        else:
            to_return.append(len(mapping))
            mapping[string] = len(mapping)
    return to_return

In [ ]:
sequencings = pd.read_csv("realdata/mutations_df_indel_event_flagged_alpha_ref.tsv", delimiter="\t")
sequencings.set_index("Unnamed: 0", inplace=True)
numerosity = pd.read_csv("realdata/Alphas_4_VirNA_Alpha_ref_node_table.tsv", delimiter="\t", usecols=["name","Nidentical"])
numerosity["numerosity"] = 1 + numerosity["Nidentical"]
numerosity.set_index("name", inplace=True)
sequencings["numerosity"] = numerosity["numerosity"]
sequencings["collection_date"] = pd.to_datetime(sequencings["collection_date"])
sequencings["days_from_beginning"] = (sequencings["collection_date"] - sequencings["collection_date"].min()).dt.days
sequencings["mutations"] = rewriteMutations(sequencings["mutations"])
sequencings["id"] = getId(sequencings["mutations"])
sequencings

In [ ]:
edgelist = pd.read_excel("realdata/Alphas_4_VirNA_Alpha_ref_edge.xlsx", usecols=["sourceID","targetID"])
edgelist

In [ ]:
sources = edgelist["sourceID"].unique()
targets = edgelist["targetID"].unique()
sources.shape, targets.shape

In [ ]:
sources[~np.isin(sources, targets)]

In [ ]:
np.any(edgelist["sourceID"] == edgelist["targetID"])

In [ ]:
edgelist.loc[edgelist["sourceID"]==97]

In [ ]:
edgelist.loc[(edgelist["sourceID"]==97)&(edgelist["targetID"]==100)]

In [ ]:
edgelist.loc[(edgelist["sourceID"]==100)&(edgelist["targetID"]==1095)]

In [ ]:
edgelist.loc[edgelist["sourceID"]==1095]

In [ ]:
edgelist.loc[edgelist["targetID"]==149]

In [ ]:
edgelist.loc[edgelist["sourceID"]==1]

In [ ]:
haplos = sequencings.sort_values("flags_day").groupby("id").first()[["mutations","flags_day","number_of_mutations","numerosity"]]
haplos

In [ ]:
haplos.loc[[1,149,1095,100,97,98,397,721,718]]

In [ ]:
haplos.loc[[95,94,740,722]]["mutations"].tolist()

In [ ]:
for id in [92,696,1378,702,701,752]:
	print(id, haplos["mutations"].loc[id], sep="\t")

In [ ]:
False < True

In [ ]:
transformed = edgelist.copy()
transformed["length"] = haplos.loc[transformed["targetID"],"number_of_mutations"].to_numpy() - haplos.loc[transformed["sourceID"],"number_of_mutations"].to_numpy()
transformed.update(
	edgelist.loc[np.logical_and(
		np.isin(edgelist["sourceID"],[1,718,722,752]),
		np.isin(edgelist["targetID"],[149,721,740,701])
	)].rename(columns={"sourceID":"targetID","targetID":"sourceID"})
)
transformed["source_uncertain"] = haplos.loc[transformed["sourceID"],"numerosity"].astype("u4").to_numpy() == 1
transformed["advance"] = haplos.loc[transformed["sourceID"],"flags_day"].to_numpy() - haplos.loc[transformed["targetID"],"flags_day"].to_numpy()
transformed

In [ ]:
treefied = transformed.sort_values(["length","source_uncertain","advance"]).groupby("targetID").first()
treefied.reset_index(inplace=True)
treefied = treefied[["sourceID","targetID"]]
treefied

In [ ]:
sources = treefied["sourceID"].unique()
targets = treefied["targetID"].unique()
sources.shape, targets.shape

In [ ]:
sources[~np.isin(sources, targets)]

In [ ]:
all_nodes = np.sort(np.unique(treefied.to_numpy()))

In [ ]:
all_nodes.shape, all_nodes

In [ ]:
np.sort(treefied["targetID"].to_numpy())

In [ ]:
np.all(np.append(all_nodes[:97], all_nodes[98:]) == np.sort(treefied["targetID"].to_numpy()))

In [ ]:
renamed = trees.Tree.renameEdgelist(treefied.to_numpy().astype("u4"), 97)

In [ ]:
renamed.shape

In [ ]:
treefied = pd.DataFrame(renamed, columns=["sourceID","targetID"])
treefied

In [ ]:
treefied.to_csv("realdata/Alpha_UK_treefied.csv", index=False)